In [53]:
import pandas as pd
import numpy as np
import ast
import datetime
import time

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
dataframe = pd.read_csv("/Users/ernestvmo/OneDrive - Universiteit Leiden/Q1/Urban Computing/Project/data/grouped_csv/data.csv", index_col=0)
dataframe.coordinates = dataframe.coordinates.apply(ast.literal_eval)

SAF_col = 'SAF_COUNT'
_311_col = '311_complaints_COUNT'
NYPD_col = 'nypd_compaints_COUNT'

#### old

In [44]:
def find_neighbours(df):
    indices = list(df['coordinates'].values)
    neighbours_all = []

    for row in df['coordinates'].values:
        x, y = row
        neighbours = []

        if x > 0:
            NW_x = x-1
            NW_y = y
            NW = (NW_x, NW_y)
            neighbours.append(('NW', NW)) if NW in indices else ...
        if x < 23:
            NE_x = x+1
            NE_y = y
            NE = (NE_x, NE_y)
            neighbours.append(('NE', NE)) if NE in indices else ...
        if y > -1:
            NN_x = x
            NN_y = y-1
            NN = (NN_x, NN_y)
            neighbours.append(('NN', NN)) if NN in indices else ...
        if y < 22:
            NS_x = x
            NS_y = y+1
            NS = (NS_x, NS_y)
            neighbours.append(('NS', NS)) if NS in indices else ...
        neighbours_all.append(neighbours)
    return neighbours_all

In [49]:
neighbs = find_neighbours(dataframe[dataframe['datestop'] == '2012-07-01'])

In [60]:
# test = dataframe[dataframe['datestop'] == '2012-07-01']
dataframe[dataframe['datestop'] == '2012-07-01'].index[dataframe[dataframe['datestop'] == '2012-07-01']['coordinates'] == (1, 20)].to_list()[0]

3

#### current

In [3]:
new_df = dataframe.copy()
new_df['NN_SAF'] = np.nan
new_df['NN_311'] = np.nan
new_df['NN_NYPD'] = np.nan

new_df['NW_SAF'] = np.nan
new_df['NW_311'] = np.nan
new_df['NW_NYPD'] = np.nan

new_df['NE_SAF'] = np.nan
new_df['NE_311'] = np.nan
new_df['NE_NYPD'] = np.nan

new_df['NS_SAF'] = np.nan
new_df['NS_311'] = np.nan
new_df['NS_NYPD'] = np.nan

new_df['SAF_lag'] = np.nan
new_df['311_lag'] = np.nan
new_df['NYPD_lag'] = np.nan

# for i,row in new_df[:10].iterrows():
    

In [4]:
new_df

,datestop,coordinates,SAF_COUNT,311_complaints_COUNT,nypd_compaints_COUNT,NN_SAF,NN_311,NN_NYPD,NW_SAF,NW_311,NW_NYPD,NE_SAF,NE_311,NE_NYPD,NS_SAF,NS_311,NS_NYPD,SAF_lag,311_lag,NYPD_lag
0,2012-07-01,"(0, 20)",0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-07-01,"(0, 21)",0.0,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-07-01,"(0, 22)",0.0,4.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-07-01,"(1, 20)",0.0,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-07-01,"(2, 20)",0.0,3.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65990,2013-06-30,"(21, 17)",0.0,1.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65991,2013-06-30,"(22, 8)",0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65992,2013-06-30,"(22, 10)",0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65993,2013-06-30,"(22, 12)",2.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def get_neighbours(indices, coordinates):
    x, y = coordinates
    neighbours = []

    if x > 0:
        NW_x = x-1
        NW_y = y
        NW = (NW_x, NW_y)
        neighbours.append(('NW', NW)) if NW in indices else ...
    if x < 23:
        NE_x = x+1
        NE_y = y
        NE = (NE_x, NE_y)
        neighbours.append(('NE', NE)) if NE in indices else ...
    if y > -1:
        NN_x = x
        NN_y = y-1
        NN = (NN_x, NN_y)
        neighbours.append(('NN', NN)) if NN in indices else ...
    if y < 22:
        NS_x = x
        NS_y = y+1
        NS = (NS_x, NS_y)
        neighbours.append(('NS', NS)) if NS in indices else ...
        
    return neighbours

here
<!-- def get_dataset_after_timelag(df: pd.DataFrame, lag):
    indices = df['coordinates'].values
    df = df.sort_values(by=["datestop"], ascending=False).reset_index(drop=True)
    datelimit = datetime.datetime.strptime(df.iloc[-1]['datestop'], "%Y-%m-%d").date()
    print(datelimit)

    # for i, row in df.iterrows():
    for (i, row), _ in zip(df.iterrows(), range(10)): # for now, don't wanna do the whole df
        row_neighbours = get_neighbours(list(indices), row['coordinates'])
        date_lag = datetime.datetime.strptime(row['datestop'], "%Y-%m-%d").date() - datetime.timedelta(days=lag)

        if datelimit > date_lag:
            break
        for neighbour in row_neighbours:
            lag_row = df[df['datestop'] == str(date_lag)][df[df['datestop'] == str(date_lag)]['coordinates'] == neighbour[1]]
            if not lag_row.empty:    
                df.at[i, f'{neighbour[0]}_SAF'] = lag_row[SAF_col].values[0]
                df.at[i, f'{neighbour[0]}_311'] = lag_row[_311_col].values[0]
                df.at[i, f'{neighbour[0]}_NYPD'] = lag_row[NYPD_col].values[0]
        print(f'{i}/{len(df)}') if (i % (len(df)/10)) == 0 else ...
    
    return df.copy() -->

In [6]:
def get_dataset_after_timelag(df: pd.DataFrame, lag):
    indices = df['coordinates'].values
    df.datestop = pd.to_datetime(df.datestop).dt.date
    df = df.sort_values(by=["datestop"], ascending=False).reset_index(drop=True)
    datelimit = df.iloc[-1]['datestop']
    # print(datelimit)

    for i, row in df.iterrows():
    # for (i, row), _ in zip(df.iterrows(), range(10)): # for now, don't wanna do the whole df
        row_neighbours = get_neighbours(list(indices), row['coordinates'])
        date_lag = row['datestop'] - datetime.timedelta(days=lag)
        # print(date_lag)
        # print(row_neighbours)


        if datelimit > date_lag:
            break
        neighbour_rows = df[df['datestop'] == row['datestop']]
        for neighbour in row_neighbours:
            # neighbour_rows = df[df['datestop'] == date_lag][df[df['datestop'] == date_lag]['coordinates'] == neighbour[1]]
            neighbour_rows = neighbour_rows[neighbour_rows['coordinates'] == neighbour[1]]
            # print(neighbour_rows.values)
            if not neighbour_rows.empty:
                df.at[i, f'{neighbour[0]}_SAF'] = neighbour_rows[SAF_col].values[0]
                df.at[i, f'{neighbour[0]}_311'] = neighbour_rows[_311_col].values[0]
                df.at[i, f'{neighbour[0]}_NYPD'] = neighbour_rows[NYPD_col].values[0]

        # lag_row = df[df['datestop'] == date_lag][df[df['datestop'] == date_lag]['coordinates'] == neighbour[1]]
            # if not lag_row.empty:    
            #     df.at[i, f'{neighbour[0]}_SAF'] = lag_row[SAF_col].values[0]
            #     df.at[i, f'{neighbour[0]}_311'] = lag_row[_311_col].values[0]
            #     df.at[i, f'{neighbour[0]}_NYPD'] = lag_row[NYPD_col].values[0]

        lag_row = df[df['datestop'] == date_lag][df[df['datestop'] == date_lag]['coordinates'] == row['coordinates']]
        if not lag_row.empty:
            df.at[i, 'SAF_lag'] = lag_row[SAF_col].values[0]
            df.at[i, '311_lag'] = lag_row[_311_col].values[0]
            df.at[i, 'NYPD_lag'] = lag_row[NYPD_col].values[0]
        
        print(f'{i}/{len(df)}') if (i % (len(df)/10)) == 0 else ...

    df = df[df['datestop'] >= (datelimit + datetime.timedelta(days=lag))]
    return df.copy()

In [54]:
data = get_dataset_after_timelag(new_df, lag=5)

0/65995
13199/65995
26398/65995
39597/65995
52796/65995


In [56]:
data = data.fillna(0)

In [59]:
data

,datestop,coordinates,SAF_COUNT,311_complaints_COUNT,nypd_compaints_COUNT,NN_SAF,NN_311,NN_NYPD,NW_SAF,NW_311,...,NE_311,NE_NYPD,NS_SAF,NS_311,NS_NYPD,SAF_lag,311_lag,NYPD_lag,x,y
0,2013-06-30,"(22, 14)",0.0,0.0,1,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,22,14
1,2013-06-30,"(12, 7)",0.0,0.0,9,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,7.0,54.0,16.0,12,7
2,2013-06-30,"(12, 15)",2.0,12.0,7,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,5.0,39.0,9.0,12,15
3,2013-06-30,"(12, 14)",9.0,10.0,15,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,14.0,64.0,22.0,12,14
4,2013-06-30,"(12, 13)",9.0,30.0,5,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,5.0,69.0,18.0,12,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65046,2012-07-06,"(16, 8)",4.0,63.0,12,0.0,0.0,0.0,0.0,29.0,...,0.0,0.0,0.0,0.0,0.0,3.0,81.0,13.0,16,8
65047,2012-07-06,"(16, 4)",11.0,66.0,28,0.0,0.0,0.0,18.0,59.0,...,0.0,0.0,0.0,0.0,0.0,3.0,40.0,21.0,16,4
65048,2012-07-06,"(16, 10)",0.0,20.0,3,0.0,0.0,0.0,0.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,4.0,16,10
65049,2012-07-06,"(16, 11)",0.0,8.0,4,0.0,0.0,0.0,2.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,16,11


In [58]:
data[['x', 'y']] = pd.DataFrame(data['coordinates'].tolist(), index=data.index)

In [67]:
test = data.copy()

In [70]:
test.datestop = pd.to_datetime(data.datestop)

In [71]:
X = data[['x', 'y', 'SAF_COUNT','311_complaints_COUNT','NN_SAF','NN_311','NN_NYPD','NW_SAF','NW_311','NW_NYPD','NE_SAF','NE_311','NE_NYPD','NS_SAF','NS_311','NS_NYPD','SAF_lag', '311_lag', 'NYPD_lag']].to_numpy()
y = data[['nypd_compaints_COUNT']].to_numpy()

### Models

#### Train/Test split

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [73]:
X_train.shape

(52040, 19)

#### GridSearch

In [ ]:
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)

In [74]:
# model = RandomForestRegressor()
# model = RandomForestRegressor()
model = GradientBoostingRegressor()

In [49]:
y_train.ravel()

array([ 3, 11,  2, ...,  3,  3,  8])

In [75]:
model = model.fit(X_train,y_train.ravel())

In [76]:
score = model.score(X_test, y_test)

In [77]:
score

0.794457812841483